# 📚 Kapitola 16: ORM - SQLAlchemy

<div style="background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h2 style="color: white; margin: 0;">Blok 2 | INTERMEDIATE</h2>
    <p style="color: white; margin: 10px 0;">📖 Výuková kapitola</p>
</div>

## 🎯 Co se naučíte

V této kapitole se zaměříme na následující témata:

- **SQLAlchemy Core vs. ORM**
- **Declarative mapping**
- **Sessions a Unit of Work**
- **Relationships: one-to-many, many-to-many**
- **Query construction**
- **Migrations with Alembic**
- **Performance optimization**

## ⚠️ Předpoklady
Tato kapitola navazuje na kapitoly: 15

---


In [ ]:
# ⚙️ Inicializace prostředí
# Tento kód nastavuje prostředí pro kapitolu 16

import sys
import os
from datetime import datetime

# Informace o prostředí
print("=" * 60)
print(f"📚 KAPITOLA 16: ORM - SQLAlchemy")
print("=" * 60)
print(f"🐍 Python verze: {sys.version}")
print(f"📅 Datum spuštění: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"💻 OS: {os.name}")
print(f"📁 Pracovní adresář: {os.getcwd()}")
print("=" * 60)

# Instalace potřebných knihoven (odkomentujte podle potřeby)
# !pip install requests pandas numpy matplotlib
# !pip install beautifulsoup4 sqlalchemy fastapi

# Import základních knihoven
import json
import csv
import time
import random
from pathlib import Path
from typing import List, Dict, Optional, Any

print("✅ Prostředí připraveno!")

## 📖 Teoretická část

<div style="background: #f0f4ff; padding: 20px; border-left: 5px solid #4a69bd; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🎓 Základní teorie a koncepty</h3>
</div>

# ORM - SQLAlchemy

## 1. Úvod a motivace

ORM (Object-Relational Mapping) je klíčová technologie při práci s databázemi v Pythonu. Umožňuje programátorům pracovat s relačními databázemi pomocí objektového modelu, bez nutnosti psát složité SQL dotazy. SQLAlchemy je jedna z nejrozšířenějších ORM knihoven v Pythonu, která podporuje jak jednoduché, tak komplexní scénáře práce s databázemi.

V praxi se SQLAlchemy používá ve všech aplikacích, které pracují s databází – od jednoduchých webových stránek až po komplexní enterprise systémy. Je součástí mnoha frameworků jako například Flask, Django nebo FastAPI, ale lze ho použít i samostatně.

Po prozkoumání této kapitoly se student naučí vytvářet a spravovat databázové modely, pracovat s relacemi mezi tabulkami, provádět dotazy, řešit problémy s výkonem a spravovat změny ve schématech databáze pomocí nástroje Alembic.

---

## 2. Hlavní koncepty

### SQLAlchemy Core vs. ORM

SQLAlchemy je rozděleno na dvě části:

1. **Core** – pokročilý nástroj pro práci s SQL. Poskytuje přímý přístup k SQL dotazům, tabulkám a schémata bez potřeby objektového modelu. Je to základ pro ORM.
2. **ORM** – objektově-relační mapování, které umožňuje pracovat s databází přes Pythonovské objekty.

Používání Core je vhodné, když potřebuješ přesně řídit SQL dotazy. ORM je ale lehčí a přehlednější pro běžné aplikace, kde se pracuje s objektovým modelem.

```python
# Příklad pomocí Core
from sqlalchemy import create_engine, Table, MetaData

engine = create_engine('sqlite:///example.db')
metadata = MetaData()
users = Table('users', metadata, autoload_with=engine)

# Příklad pomocí ORM
from sqlalchemy.orm import declarative_base, sessionmaker

Base = declarative_base()
Session = sessionmaker(bind=engine)
```

---

### Declarative mapping

Declarative mapping je způsob, jakým definuješ databázové tabulky jako Pythonovské třídy. Každá třída představuje jednu tabulku, a její atributy odpovídají sloupcům.

```python
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import declarative_base

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    email = Column(String(100))

# Tato třída se pak převede na tabulku v databázi
```

Použití `declarative_base()` je klíčové – umožňuje SQLAlchemy poznat, že třídy mají být mapovány na databázové tabulky.

---

### Sessions a Unit of Work

Session je hlavní objekt pro práci s databází v SQLAlchemy. Slouží k ukládání změn do databáze a jejich řízení (např. commit, rollback). V rámci jedné session se udržuje sled změn – tedy Unit of Work pattern.

```python
from sqlalchemy.orm import Session

session = Session(engine)

user = User(name="Jan", email="jan@example.com")
session.add(user)
session.commit()  # Změny se uloží do databáze
```

Session také umožňuje načítání objektů z databáze a jejich manipulaci:

```python
user = session.query(User).filter_by(name="Jan").first()
user.email = "jan.new@example.com"
session.commit()
```

---

### Relationships: one-to-many, many-to-many

V relačních databázích se vztahy mezi tabulkami reprezentují pomocí foreign keys. SQLAlchemy umožňuje definovat tyto vztahy mezi objekty.

#### One-to-Many

Příklad: Jedna uživatel může mít více příspěvků.

```python
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    posts = relationship("Post", back_populates="user")

class Post(Base):
    __tablename__ = 'posts'
    id = Column(Integer, primary_key=True)
    title = Column(String(100))
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="posts")
```

#### Many-to-Many

Vztah mezi dvěma entitami, kde každá entita může mít více souvisejících objektů druhého typu.

```python
# Tabulka propojovací
tags = Table('post_tags', Base.metadata,
    Column('post_id', Integer, ForeignKey('posts.id')),
    Column('tag_id', Integer, ForeignKey('tags.id'))
)

class Post(Base):
    __tablename__ = 'posts'
    id = Column(Integer, primary_key=True)
    title = Column(String(100))
    tags = relationship("Tag", secondary=tags, back_populates="posts")

class Tag(Base):
    __tablename__ = 'tags'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    posts = relationship("Post", secondary=tags, back_populates="tags")
```

---

### Query construction

SQLAlchemy umožňuje vytvářet dotazy pomocí Pythonovské syntaxe. Můžeš použít `query()` metodu a řetězit podmínky.

```python
# Načtení všech uživatelů
users = session.query(User).all()

# Filtrace podle jména
user = session.query(User).filter_by(name="Jan").first()

# Složitější dotaz
users = session.query(User).filter(
    User.name.like("%Jan%")
).order_by(User.id.desc())
```

Můžeš také použít SQLAlchemy Core syntaxi:

```python
from sqlalchemy import select

stmt = select(User).where(User.name == "Jan")
result = session.execute(stmt)
```

---

### Migrations with Alembic

Migrace jsou důležité, když se schéma databáze mění v průběhu vývoje aplikace. Alembic je nástroj pro správu verzí databázových schémat.

```bash
alembic init alembic  # Inicializace
alembic revision --autogenerate -m "Add new table"  # Vytvoření migrace
alembic upgrade head  # Aplikace migrací
```

Alembic automaticky detekuje změny ve třídách modelů a vytvoří odpovídající SQL skripty.

---

### Performance optimization

SQLAlchemy umožňuje optimalizovat výkon různými způsoby:

- **Lazy loading** vs. **Eager loading**: Nastavením `lazy="joined"` nebo `lazy="select"` ovlivníš, jak se načítají vazby.
  
```python
posts = relationship("Post", back_populates="user", lazy="joined")
```

- **Batch operations** pro větší množství záznamů:

```python
session.bulk_insert_mappings(User, [{'name': 'Jan'}, {'name': 'Petr'}])
```

- **Use of `yield_per()`** pro velké soubory (např. při exportu):

```python
for user in session.query(User).yield_per(1000):
    print(user.name)
```

---

## 3. Důležité detaily

### Časté chyby a jak se jim vyhnout

- **Nepoužívání `session.commit()`** – změny se neuloží.
- **Použití `query().all()` na velkých datech** – může způsobit přetížení paměti.
- **Zapomnět na `relationship` v modelu** – chyby při načítání vazeb.

### Best practices

- Vždy používej session context manager nebo explicitní `close()`.
- Definuj `__repr__` pro lehčí ladění.
- Používej `lazy="select"` nebo `lazy="joined"` podle potřeby.

### Performance tipy

- Při práci s velkými objekty používej `session.query(User).options(lazy="select")`.
- Dávej pozor na N+1 problémy – zkontroluj, jestli se nevolají dotazy uvnitř cyklu.
- Používej `bulk_insert_mappings()` pro hromadné vkládání.

---

## 4. Propojení s předchozími kapitolami

Tato kapitola je logickým pokračováním předchozích kapitol o databázích a SQL. Využívá základní znalosti práce s SQL dotazy, tabulkami a relacemi. Přechod od jednoduchých SQL příkazů k ORM umožňuje programátorům psát čistší a znovupoužitelnější kód.

Naučené koncepty jako práce s relacemi, dotazy a indexy se rozšiřují v tomto modelu o objektový přístup a správu stavu. Tato kapitola tedy přináší praxi zpět do aplikací a ukazuje, jak se ORM používá ve skutečných projektech.

---

**Závěr:** ORM v SQLAlchemy je mocný nástroj, který umožňuje psát Pythonovský kód místo SQL. Umíš vytvářet modely, pracovat s vazbami, spravovat změny a optimalizovat výkon – vše v rámci jednoduchého a přehledného API.


## 💻 Praktické příklady

<div style="background: #e8f5e9; padding: 20px; border-left: 5px solid #4caf50; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">👨‍💻 Hands-on příklady ke spuštění</h3>
    <p>Následující příklady si můžete hned vyzkoušet. Každý příklad je samostatně spustitelný.</p>
</div>


# 1. SQLAlchemy Core vs. ORM

**Co demonstruje:** Tento příklad porovnává způsob práce s SQLAlchemy jako Core (prostý SQL) a jako ORM (objektové mapování).

```python
from sqlalchemy import create_engine, text, Column, Integer, String, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Nastavení databáze (použijeme SQLite pro jednoduchost)
engine = create_engine('sqlite:///example.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

# Definice tabulek pomocí SQLAlchemy Core
users_table = Table('users', Base.metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String(50)),
    Column('email', String(100))
)

posts_table = Table('posts', Base.metadata,
    Column('id', Integer, primary_key=True),
    Column('title', String(100)),
    Column('user_id', Integer, ForeignKey('users.id'))
)

# Definice modelů pomocí ORM
class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    email = Column(String(100))

class Post(Base):
    __tablename__ = 'posts'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(100))
    user_id = Column(Integer, ForeignKey('users.id'))

# Vytvoření tabulek
Base.metadata.create_all(engine)

# ORM přístup - vkládání dat
session = Session()
user_orm = User(name="Jan Novák", email="jan.novak@example.com")
session.add(user_orm)
session.commit()

# Core přístup - vkládání dat
with engine.connect() as conn:
    # Vložení uživatele pomocí Core
    result = conn.execute(text("INSERT INTO users (name, email) VALUES (:name, :email)"),
                         {"name": "Petr Svoboda", "email": "petr.svoboda@example.com"})
    conn.commit()

# ORM přístup - dotazování dat
users_orm = session.query(User).all()
print("ORM výsledky:")
for user in users_orm:
    print(f"ID: {user.id}, Jméno: {user.name}, Email: {user.email}")

# Core přístup - dotazování dat
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM users"))
    print("\nCore výsledky:")
    for row in result:
        print(f"ID: {row[0]}, Jméno: {row[1]}, Email: {row[2]}")

session.close()
```

**Očekávaný výstup:**
```
ORM výsledky:
ID: 1, Jméno: Jan Novák, Email: jan.novak@example.com
ID: 2, Jméno: Petr Svoboda, Email: petr.svoboda@example.com

Core výsledky:
ID: 1, Jméno: Jan Novák, Email: jan.novak@example.com
ID: 2, Jméno: Petr Svoboda, Email: petr.svoboda@example.com
```

**Vysvětlení:** Tento příklad ukazuje dvě různé cesty práce s databází v SQLAlchemy. ORM (Object-Relational Mapping) umožňuje pracovat s objekty Pythonu, zatímco Core poskytuje přímý přístup k SQL příkazům. ORM je komfortnější pro vývoj, zatímco Core nabízí více kontroly nad SQL dotazy.

---

# 2. Declarative mapping

**Co demonstruje:** Tento příklad ukazuje, jak definovat modely pomocí deklarativního mapování a vytvářet vazby mezi tabulkami.

```python
from sqlalchemy import create_engine, Column, Integer, String, DateTime, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Nastavení databáze
engine = create_engine('sqlite:///example2.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

# Definice modelů pomocí deklarativního mapování
class Author(Base):
    __tablename__ = 'authors'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    email = Column(String(120), unique=True)
    created_at = Column(DateTime, default=func.now())
    
    def __repr__(self):
        return f"<Author(id={self.id}, name='{self.name}', email='{self.email}')>"

class Book(Base):
    __tablename__ = 'books'
    
    id = Column(Integer, primary_key=True)
    title = Column(String(200), nullable=False)
    isbn = Column(String(13), unique=True)
    author_id = Column(Integer, ForeignKey('authors.id'))
    
    # Vztah k autorovi
    author = relationship("Author", back_populates="books")
    
    def __repr__(self):
        return f"<Book(id={self.id}, title='{self.title}', isbn='{self.isbn}')>"

# Vytvoření tabulek
Base.metadata.create_all(engine)

# Vytvoření sezení
session = Session()

# Vložení dat
author1 = Author(name="George Orwell", email="orwell@example.com")
author2 = Author(name="Aldous Huxley", email="huxley@example.com")

session.add_all([author1, author2])
session.commit()

print("Vytvoření autorů:")
print(author1)
print(author2)

# Vložení knih
book1 = Book(title="1984", isbn="9780451524935", author_id=author1.id)
book2 = Book(title="Brave New World", isbn="9780060850524", author_id=author2.id)

session.add_all([book1, book2])
session.commit()

print("\nVytvoření knih:")
print(book1)
print(book2)

# Uzavření sezení
session.close()
```

**Očekávaný výstup:**
```
Vytvoření autorů:
<Author(id=1, name='George Orwell', email='orwell@example.com')>
<Author(id=2, name='Aldous Huxley', email='huxley@example.com')>

Vytvoření knih:
<Book(id=1, title='1984', isbn='9780451524935')>
<Book(id=2, title='Brave New World', isbn='9780060850524')>
```

**Vysvětlení:** V tomto příkladu je použit deklarativní přístup k mapování objektů do databázových tabulek. Definujeme třídy `Author` a `Book`, které odpovídají tabulkám v databázi. Používáme `ForeignKey` pro vytvoření vazby mezi tabulkami a `relationship` pro definici relace mezi objekty.

---

# 3. Sessions a Unit of Work

**Co demonstruje:** Tento příklad ukazuje, jak funguje koncept Unit of Work pomocí SQLAlchemy session.

```python
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Nastavení databáze
engine = create_engine('sqlite:///example3.db', echo=True)
Base = declarative_base()
Session = sessionmaker(bind=engine)

class Product(Base):
    __tablename__ = 'products'
    
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    price = Column(Integer)  # Cena v haléřích

# Vytvoření tabulky
Base.metadata.create_all(engine)

# Vytvoření session
session = Session()

# Přidání produktů do session (Unit of Work)
product1 = Product(name="Laptop", price=15000)
product2 = Product(name="Mouse", price=200)
product3 = Product(name="Keyboard", price=800)

session.add(product1)
session.add(product2)
session.add(product3)

# Zatím nejsou v databázi - jsou jen v session
print("Přidání produktů do session (zatím neuloženo):")
print(f"Počet objektů v session: {len(session.new)}")

# Uložení změn do databáze
session.commit()
print("Změny byly uloženy do databáze")

# Změna existujícího produktu
product = session.query(Product).filter_by(name="Laptop").first()
product.price = 14000  # Snížení ceny

print(f"Cena po změně: {product.price}")

# Vložení dalších změn do session
session.add(product)

# Commit - změny se uloží do databáze
session.commit()
print("Změna ceny byla uložena")

# Zrušení změn (rollback)
product = session.query(Product).filter_by(name="Mouse").first()
old_price = product.price
product.price = 300

print(f"Původní cena: {old_price}, Nová cena: {product.price}")

# Vrácení změn zpět
session.rollback()  # Zruší změny v session
print(f"Cena po rollbacku: {product.price}")

# Uzavření session
session.close()
```

**Očekávaný výstup:**
```
Přidání produktů do session (zatím neuloženo):
Počet objektů v session: 3
Změny byly uloženy do databáze
Cena po změně: 14000
Změna ceny byla uložena
Původní cena: 200, Nová cena: 300
Cena po rollbacku: 200
```

**Vysvětlení:** Session je hlavním objektem SQLAlchemy pro práci s databází. V rámci jedné session se ukládají změny (Unit of Work pattern). Změny jsou provedeny až po volání `commit()`. Pokud chceme vrátit změny zpět, použijeme `rollback()`.

---

# 4. Relationships: one-to-many

**Co demonstruje:** Tento př

## 🎯 Cvičení a úkoly

<div style="background: #fff3e0; padding: 20px; border-left: 5px solid #ff9800; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">✍️ Praktická cvičení k procvičení</h3>
    <p>Vyřešte následující úkoly. Začněte od jednoduššších a postupujte k složitějším.</p>
</div>

# Cvičení pro kapitolu: ORM - SQLAlchemy

---

## 1. **Název úkolu**: Vytvoření modelu a základní CRUD operace  

### **Detailní zadání**  
Vytvoř tabulku `User` s následujícími sloupci: `id`, `name`, `email`, `age`. Napiš SQLAlchemy model pro tuto tabulku a implementuj funkce pro základní CRUD operace: vytvoření uživatele, načtení uživatele podle ID, aktualizaci uživatele a smazání uživatele.

### **Vstupní data/požadavky**  
- Použij SQLAlchemy ORM
- Vytvoř model `User` s výše uvedenými sloupci
- Zajisti správné typy dat (např. `String`, `Integer`)
- Implementuj funkce pro: create, read, update, delete

### **Očekávaný výstup**  
- Model `User` definovaný pomocí SQLAlchemy ORM
- Funkce: `create_user`, `get_user`, `update_user`, `delete_user`
- Všechny operace mají být provedeny přes session

### **Hints/Nápověda**
1. Použij `declarative_base()` a vytvoř třídu `User`, která dědí od `Base`.
2. Definuj sloupce jako atributy třídy s typem (např. `Column(String(50))`).
3. Pro operace využij session (`session.add()`, `session.query()`, atd.).
4. Ujisti se, že používáš `session.commit()` po provedení změn.

### **Kostra řešení**
```python
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    email = Column(String(100))
    age = Column(Integer)

# Napiš funkce: create_user, get_user, update_user, delete_user
```

### **Bonusové rozšíření**  
Přidej validaci e-mailu při vytváření uživatele (např. pomocí regulárního výrazu).

---

## 2. **Název úkolu**: Relace mezi tabulkami – One-to-Many  

### **Detailní zadání**  
Vytvoř dvě modely: `User` a `Post`. Každý uživatel může mít více příspěvků. Vytvoř relaci „jeden k mnoha“ mezi nimi. Implementuj funkce pro přidání nového příspěvku ke konkrétnímu uživateli a načtení všech příspěvků uživatele.

### **Vstupní data/požadavky**  
- Vytvoř model `Post` s vazbou na `User`
- Použij `relationship()` a `ForeignKey()` k vytvoření vazby
- Implementuj funkce: přidání příspěvku, načtení příspěvků uživatele

### **Očekávaný výstup**  
- Modely `User` a `Post` s relací
- Funkce pro přidání příspěvku a načtení příspěvků uživatele

### **Hints/Nápověda**
1. V modelu `Post` přidej sloupec `user_id` jako `ForeignKey('users.id')`.
2. V modelu `User` použij `relationship("Post", back_populates="user")`.
3. Při vytváření příspěvku můžeš použít `session.add(post)` nebo přiřadit objekt přímo.
4. Využij `session.query(Post).filter(Post.user_id == user_id)`

### **Kostra řešení**
```python
class User(Base):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    posts = relationship("Post", back_populates="user")

class Post(Base):
    __tablename__ = 'posts'
    id = Column(Integer, primary_key=True)
    title = Column(String(100))
    content = Column(String(500))
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="posts")
```

### **Bonusové rozšíření**  
Přidej funkci pro výpočet počtu příspěvků uživatele.

---

## 3. **Název úkolu**: Použití SQLAlchemy filtrů a agregací  

### **Detailní zadání**  
Vytvoř dotaz, který vrátí všechny uživatele starší než 25 let. Dále přidej dotaz, který spočítá průměrný věk uživatelů. Využij SQLAlchemy query API.

### **Vstupní data/požadavky**  
- Použij `session.query()`
- Filtruj podle `age > 25`
- Spočítej průměr věku pomocí `func.avg()`

### **Očekávaný výstup**  
- Výpis uživatelů starších než 25 let
- Průměrný věk všech uživatelů

### **Hints/Nápověda**
1. Použij `session.query(User).filter(User.age > 25)`
2. Využij `func.avg(User.age)` z modulu `sqlalchemy`
3. Ujisti se, že přidáš `session.commit()` pokud to potřebuješ
4. Můžeš použít `.all()`, `.first()`, nebo `.scalar()` podle typu výstupu

### **Kostra řešení**
```python
from sqlalchemy import func

# Napiš dotazy pro:
# - uživatele > 25 let
# - průměrný věk
```

### **Bonusové rozšíření**  
Vytvoř dotaz, který vrátí počet uživatelů podle věku (např. skupiny 20-30, 30-40 atd.).

---

## 4. **Název úkolu**: Transakce a chybové zpracování  

### **Detailní zadání**  
Vytvoř funkci, která přidá dvě nové uživatele do databáze v rámci jedné transakce. Pokud se nějaký příkaz nepodaří (např. duplicitní email), transakce se zruší. Implementuj chybové zpracování a logování.

### **Vstupní data/požadavky**  
- Vytvoř funkci s dvěma `session.add()` příkazy
- Použij `try/except` pro zachycení chyb
- Zajisti, že transakce se ruší v případě chyby

### **Očekávaný výstup**  
- V případě úspěchu: oba uživatelé přidáni
- V případě chyby: žádný z uživatelů nebyl přidán, chyba logována

### **Hints/Nápověda**
1. Použij `session.begin()` pro ruční začátek transakce.
2. Využij `session.rollback()` v případě chyby.
3. Můžeš použít `try-except` a kontrolovat typ výjimky (např. `IntegrityError`).
4. Nezapomeň zavolat `session.commit()` pouze pokud nenastala chyba.

### **Kostra řešení**
```python
from sqlalchemy.exc import IntegrityError

def add_users_safely():
    try:
        # Přidání uživatelů
        session.add(user1)
        session.add(user2)
        session.commit


## 📚 Další zdroje a materiály

<div style="background: #e3f2fd; padding: 20px; border-left: 5px solid #2196f3; margin: 20px 0; border-radius: 5px;">
    <h3 style="color: #2c3e50; margin-top: 0;">🔗 Doporučené materiály k dalšímu studiu</h3>
</div>

# ORM – SQLAlchemy – Doporučené zdroje

## 📄 Články a tutoriály

1. **[SQLAlchemy Documentation - The Tutorial](https://docs.sqlalchemy.org/en/20/tutorial/index.html)**  
   Oficiální úvodní průvodce SQLAlchemy, který vás seznámí s hlavními koncepty jako jsou modelování, relace, query a transakce. Skvělé pro začátečníky.

2. **[Real Python - SQL with SQLAlchemy](https://realpython.com/python-sqlite-orm/)**  
   Podrobný článek o použití SQLAlchemy v praxi, ukazuje, jak pracovat s databázemi, modelovat data a provádět dotazy.

3. **[Full Stack Python - SQLAlchemy](https://www.fullstackpython.com/sqlalchemy.html)**  
   Shrnutí a praktické příklady použití SQLAlchemy v rámci webových aplikací. Vhodné pro vývojáře, kteří chtějí využít ORM v rámci frameworku Flask nebo Django.

4. **[Python ORM with SQLAlchemy and Flask](https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-iv-database)**  
   Široký průvodce vytvářením databázových aplikací s použitím SQLAlchemy a Flasku. Vysvětluje i napojení na PostgreSQL a SQLite.

5. **[SQLAlchemy 2.0 Migration Guide](https://docs.sqlalchemy.org/en/20/changelog/migration_20.html)**  
   Pokud používáš SQLAlchemy 2.0, tento článek ti pomůže přejít od starší verze a pochopit změny v API a syntaxi.

---

## 🎥 YouTube videa

1. **[SQLAlchemy Tutorial for Beginners - Python ORM](https://www.youtube.com/watch?v=5x72Q38dX4s)** (45 min)  
   Zábavný a stručný úvod do SQLAlchemy pro začátečníky – vysvětluje základy modelování, relací a query.

2. **[Flask + SQLAlchemy - Full Course](https://www.youtube.com/watch?v=4017D2X9w0k)** (1h 30 min)  
   Komplexní kurz zaměřený na použití SQLAlchemy v aplikacích založených na Flasku, ideální pro webové vývojáře.

3. **[Python ORM - SQLAlchemy vs Django ORM](https://www.youtube.com/watch?v=7gX6VdQp1zU)** (30 min)  
   Porovnání SQLAlchemy s Django ORM – vysvětluje, kdy je lepší použít jednu nebo druhou.

---

## 📚 Knihy a oficiální dokumentace

1. **[SQLAlchemy 2.0 Documentation](https://docs.sqlalchemy.org/en/20/)**  
   Nejkompletnější a nejaktuální oficiální dokumentace, která obsahuje příklady, průvodce a detailní popis funkcí SQLAlchemy.

2. **[Flask Web Development - Miguel Grinberg](https://www.oreilly.com/library/view/flask-web-development/9781491939344/)**  
   Knížka věnovaná tvorbě webových aplikací v Pythonu – zahrnuje kapitolu o SQLAlchemy a práci s databázemi.

3. **[Effective SQLAlchemy - Rick Copeland](https://www.oreilly.com/library/view/effective-sqlalchemy/9781491963022/)**  
   Vývojářská příručka zaměřená na efektivní použití SQLAlchemy, která se zaměřuje na pokročilé techniky a tipy na optimalizaci.

---

## 💻 Praktické projekty

1. **Blogová aplikace s Flaskem a SQLAlchemy**  
   Vytvoř jednoduchý blog s možností přidá


## 📝 Shrnutí kapitoly

<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin: 30px 0;">
    <h3 style="color: white; margin-top: 0;">✅ Co jste se naučili</h3>
    <ul style="list-style: none; padding-left: 0;">
        <li>✓ SQLAlchemy Core vs. ORM</li>
<li>✓ Declarative mapping</li>
<li>✓ Sessions a Unit of Work</li>
<li>✓ Relationships: one-to-many, many-to-many</li>
<li>✓ Query construction</li>
    </ul>
    
    <h3 style="color: white;">🎯 Klíčové dovednosti</h3>
    <p>Po dokončení této kapitoly byste měli být schopni prakticky použít všechny probrané koncepty.</p>
    
    <h3 style='color: white;'>➡️ Další kapitola</h3><p>Kapitola 17 - pokračujte ve studiu!</p>
</div>

---

*📅 Notebook vygenerován: 2025-09-29 12:25:40*  
*🤖 Generátor: Comprehensive Colab Generator v2.0*  
*📚 Učebnice programování - Od základů k AI*


## 🧪 Sandbox - Prostor pro experimenty

Použijte následující buňky pro vlastní experimenty a testování:

In [ ]:
# 🧪 Zde můžete experimentovat s kódem z kapitoly
# Napište svůj kód zde:

